In [1]:
import pandas as pd
from numpy import array
import math
import csv

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint


from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession

In [2]:
conf=SparkConf().setMaster("local").setAppName("SparkDecsisionTree2")
sc=SparkContext(conf=conf)

In [4]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)#Initialize spark

df=pd.read_csv('player-stat.csv')
s_df=sqlContext.createDataFrame(df)

In [5]:
train_dataset  = s_df.rdd.map(lambda x: LabeledPoint(x[9], x[2:9]))

model_2=DecisionTree.trainRegressor(train_dataset,categoricalFeaturesInfo={},impurity='variance',maxDepth=30,maxBins=128)

model_2.save(sc,"model1")
#End of Decision Tree

In [6]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from numpy import array
model = DecisionTreeModel.load(sc, "model1")

In [7]:
#Smaple test case
test_1=[array([43.63,126.64,33,13.2,15,6.3,1])]
test_1=sc.parallelize(test_1)
predicitons_1=model.predict(test_1)
predicitons_1.collect()

[nan]

In [ ]:
import random
import csv
import pandas as pd  

bat_clust = dict()
bowl_clust = dict()
team1 = []
team2 = []
t1=[]
t2=[]
'''
files = pd.read_csv('step2/player_to_player.csv')  #Reading from the probability file 
for i in range(len(files)):
    batname = files.loc[i].batsman.strip()     #Keys of dictionary are batsman name and values are cluster number
    bat_clust[batname] = files.loc[i].batclustno
    bowlname = files.loc[i].bowler.strip()   #Keys of dictionary are bowler name and values are cluster number
    bowl_clust[bowlname] = files.loc[i].bowlclustno
'''


#bowl_clust['SS Tiwary'] = 0

with open('/home/shashi/Documents/big-data/squad1.csv') as csvFile:
    reader = csv.reader(csvFile)

    for r in reader:
        t1.append(r[0])
        t2.append(r[1])
        break

    reader=csv.reader(csvFile)
    next(reader)
    for row1 in reader:
        team1.append(row1[0])
        team2.append(row1[1])
csvFile.close()

#clust = pd.read_csv('step2/clust_to_clust.csv')


def bowlname(bowler):
    newname2=bowler
    with open('/home/shashi/Documents/step-3/player-stat.csv') as hd:
        player_stat = csv.reader(hd)
        for row in player_stat:
            if (newname2==row[1]):
                avg2 = row[4]
                eco2 = row[5]
                strike2 = row[6]
                print(avg2,eco2,strike2)
                return avg2,eco2,strike2

def batname(batsman):
    newname = batsman
    with open('player-stat.csv') as hd:
        player_stat = csv.reader(hd)
        for row in player_stat:
            if (newname == row[0]):
                ip_avg = row[2]
                ip_strike = row[3]
                print(ip_avg,ip_strike)
                return ip_avg,ip_strike
        
def playerprob(batsman,bowler,innings,o_ball):
    with open('/home/shashi/Documents/step-3/player-stat.csv') as hd:
        player_stat = csv.reader(hd)
        for row in player_stat:
            if (batsman == row[0]):
                ip_avg = row[2]
                ip_strike = row[3]
                print(ip_avg,ip_strike)
                break#return ip_avg,ip_strike
                
    with open('/home/shashi/Documents/step-3/player-stat.csv') as hd:
        player_stat = csv.reader(hd)
        for row in player_stat:
            if (bowler==row[1]):
                avg2 = row[4]
                eco2 = row[5]
                strike2 = row[6]
                print(avg2,eco2,strike2)
                break#return avg2,eco2,strike2
    
    ba_av,ba_sr=ip_avg,ip_strike
    print(ba_av,ba_sr)
    bo_av,bo_eco,bo_sr=avg2,eco2,strike2
    print(bo_av,bo_eco,bo_sr)

    test_1=[array([ba_av,ba_sr,bo_av,bo_eco,bo_sr,o_ball,innings])]
    test_1=sc.parallelize(test_1)
    predicitons_1=model.predict(test_1).collect()
    return predicitons_1[0]


def clusterprob(bat_clust_no,bowl_clust_no):
    #row = clust_prob.loc[clust_prob['batclustno'] == bat_clust_no].loc[clust_prob['bowlclustno'] == bowl_clust_no]
    
    row = player_stat.loc[player_stat[0] == batsman].loc[player_stat[1] == bowler]

    #ba_av,ba_sr = bat_function(batsman)
    ba_av,ba_sr = row[2],row[3]

    #bo_av,bo_eco,bo_sr = bow_function(bowler)
    bo_av,bo_eco,bo_sr = row[4],row[5],row[6]

    test_1=[array([innings,o_ball,ba_av,ba_sr,bo_av,bo_eco,bo_sr])]
    test_1=sc.parallelize(test_1)
    predicitons_1=model.predict(test_1).collect()
    return round(predicitons_1[0])




def innings1(team1, team2):
    innings=1
    striker = team1[0]
    non_striker = team1[1]
    bowler = team2[len(team2)-1]
    nextbatsman = 2
    wick = 0
    runs = 0
    overs = 0

    count = 2

    dic = {}


    while(overs<20 and wick < 10):
        balls = 1
        cl = 0
        while(balls<7 and wick <10):
            #try:
                #row = player_stat.loc[player_stat['batsman'] == striker].loc[player_stat['bowler'] == bowler]
            '''except:
                row = clust_prob.loc[clust_prob['batclustno'] == bat_clust[striker]].loc[clust_prob['bowlclustno'] == bowl_clust[bowler]]
                cl = 1
            '''	
            score = 0
            flag = 0
            o_ball=float(str(overs)+'.'+str(balls))
            if cl==0:
                #if (int(row['balls'])>=15):
                print(striker,bowler)
                score = playerprob(striker,bowler,innings,o_ball)
                print(score)
                #else:
                    #score = clustrprob(bat_clust[striker],bowl_clust[bowler])
                    #score = playerprob(striker,bowler,innings,o_ball)
                        
            else:
                    #score = clustprob(bat_clust[striker],bowl_clust[bowler])
                score = playerprob(striker,bowler,innings,o_ball)

            if(score==7):
                wick=wick+1
                if(wick<9):
                    striker = team1[nextbatsman]
                    nextbatsman = (nextbatsman+1)%11
                flag = 1
            if(flag==0):
                runs = runs+score
                if (score==1 or score == 3):
                    striker,non_striker = non_striker,striker
            balls = balls+1
        striker,non_striker = non_striker,striker                 
        bowler = team2[len(team2)-count]                    
        count = (count+1)%5 + 1 
        overs = overs+1
        print(overs)
    return runs,wick


def innings2(team1, team2,runs1):
    striker = team1[0]
    non_striker = team1[1]
    bowler = team2[len(team2)-1]
    nextbatsman = 2
    wick = 0
    runs = 0
    overs = 0
    prob = 0
    count = 2
    innings=2
    dic = {}
    while(overs<20 and wick < 10):
        balls = 1
        cl = 0
        
        while(balls<7 and runs<=runs1 and wick <10):
            '''try:
                row = player_prob.loc[player_prob['batsman'] == striker].loc[player_prob['bowler'] == bowler]
                prob = float(row['notout'])
            except:
                row = clust_prob.loc[clust_prob['batclustno'] == bat_clust[striker]].loc[clust_prob['bowlclustno'] == bowl_clust[bowler]]
                prob = float(row['notout'])
                cl = 1
            '''	
            score = 0
            flag = 0
            #print(striker,dic[striker])
            o_ball=float(str(overs)+'.'+str(balls))
            if cl==0:
                #if (int(row['balls'])>=15):
                score = playerprob(striker,bowler,innings,o_ball)
                #else:
                    #score = clusterprob(bat_clust[striker],bowl_clust[bowler])
                    #score = playerprob(striker,bowler,innings,o_ball)
                        
            else:
                #print(striker,bowler)
                #score = clusterprob(bat_clust[striker],bowl_clust[bowler])
                score = playerprob(striker,bowler,innings,o_ball)
            
            if(score==7):
                wick = wick+1
                if(wick<9):
                    striker = team1[nextbatsman]
                    nextbatsman = (nextbatsman+1)%11
                flag = 1
            if(flag==0):
                runs = runs+score
                if (score==1 or score == 3):
                    striker,non_striker = non_striker,striker
            balls = balls+1
        striker,non_striker = non_striker,striker                 
        bowler = team2[len(team2)-count]                    
        count = (count+1)%5 + 1 
        overs = overs+1
        print(overs)
        if (runs>runs1):
            break
    return runs,wick


runs1 ,wicks1 = innings1(team1,team2)
runs2 ,wicks2 = innings2(team2,team1,runs1)
print()
print("Score of innings 1")
print(runs1,wicks1)
print()
print("Score of innings 2")
print(runs2,wicks2)
print()
res1=runs1-runs2
res2=10-wicks2

if(runs1>runs2):
    print(t1[0],"has won by",res1,"runs")
elif(runs1<runs2):
    print(t2[0],"has won by",res2,"wickets")
else:
    print("Tie!!!")
